In [72]:
import spacy
from spacy.tokens import Token
from spacy import displacy
import pandas as pd
import pickle
import random
import tqdm
import numpy as np

In [2]:
all_verbs = []

In [3]:
file = open("../stored_data/docs.obj",'rb')
docs = pickle.load(file)
file.close()

In [4]:
id = random.sample(list(docs.keys()), 1)
random_doc = docs[id[0]]
full_doc = ""
for sent in random_doc:
    full_doc += sent

In [5]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_lg")
entities = nlp.get_pipe('ner').labels

In [6]:
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [
    {"ENT_TYPE": {'IN': entities}}, 
    {"ENT_TYPE": {'NOT_IN': entities}, 'OP': '*'}, 
    {'POS': 'VERB'},
    {"ENT_TYPE": {'NOT_IN': entities}, 'OP': '*'}, 
    {"ENT_TYPE": {'IN': entities}}]
matcher.add("any_verb", [pattern])

In [17]:
doc = nlp("Fairchild Corp was acquired by Fujitsu. Fairchild Corp acquired Fujitsu.")

matches = matcher(doc)
e2e_relationship = []


for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

    is_passive = False

    for token in span:
        if token.pos_ == "VERB":
            verb = token.text
            #all_verbs.append(token)
        if "pass" in token.dep_:
            is_passive = True

    if is_passive:
        print('here')
        for token in span:
            if "subjpass" in token.dep_:
                obj = token
                obj_ent = token.ent_type_
            elif "obj" in token.dep_:
                subj = token
                subj_ent = token.ent_type_
    else:
        for token in span:
            if "obj" in token.dep_:
                obj = token
                obj_ent = token.ent_type_
            elif "subj" in  token.dep_:
                subj = token
                subj_ent = token.ent_type_


    e2e_relationship.append([subj, subj_ent, verb, obj, obj_ent])


print(e2e_relationship)

13200282023714757851 any_verb 1 6 Corp was acquired by Fujitsu
here
13200282023714757851 any_verb 8 11 Corp acquired Fujitsu
[[Fujitsu, 'ORG', 'acquired', Corp, 'ORG'], [Corp, 'ORG', 'acquired', Fujitsu, 'ORG']]


In [22]:
all_verbs = []
e2e_relationship = []

for id, file in tqdm.tqdm(docs.items()):
    full_doc = ""
    for sent in file:
        full_doc += sent

    doc = nlp(full_doc)

    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span

        is_passive = False

        for token in span:
            if token.pos_ == "VERB":
                verb = token.text
                all_verbs.append(token)
            if "pass" in token.dep_:
                is_passive = True

        if is_passive:
            for token in span:
                if "subjpass" in token.dep_:
                    obj = token
                    obj_ent = token.ent_type_
                elif "obj" in token.dep_:
                    subj = token
                    subj_ent = token.ent_type_
        else:
            for token in span:
                if "obj" in token.dep_:
                    obj = token
                    obj_ent = token.ent_type_
                elif "subj" in  token.dep_:
                    subj = token
                    subj_ent = token.ent_type_


        e2e_relationship.append([subj, subj_ent, verb, obj, obj_ent])
        

100%|██████████| 8399/8399 [03:43<00:00, 37.63it/s]


In [28]:
e2e_relationship

[[EXPORTERS, '', 'fear', damage, ''],
 [EXPORTERS, '', 'raise', many, ''],
 [saythey, '', 'tell', businessman, ''],
 [saythey, '', 'boost', US, 'GPE'],
 [saythey, '', 'curb', US, 'GPE'],
 [conflict, '', 'hurt', term, ''],
 [it, '', 'impose', term, ''],
 [it, '', 'sell', market, ''],
 [it, '', 'put', tariff, ''],
 [spokesman, '', 'say', business, ''],
 [tariff, '', 'remain', time, ''],
 [it, '', 'mean', US, 'GPE'],
 [it, '', 'say', US, 'GPE'],
 [it, '', 'ltjame', broker, ''],
 [we, '', 'say', warning, ''],
 [who, '', 'have', surplus, ''],
 [pct, '', 'swell', help, ''],
 [pct, '', 'allow', import, ''],
 [we, '', 'defuse', problem, ''],
 [we, '', 'say', retaliation, ''],
 [dispute, '', 'say', US, 'GPE'],
 [dispute, '', 'lead', pressure, ''],
 [dispute, '', 'have', surplus, ''],
 [curb, '', 'say', Japan, 'GPE'],
 [curb, '', 'hit', semiconductor, ''],
 [curb, '', 'expand', US, 'GPE'],
 [Japan, 'GPE', 'allege', Kong, 'GPE'],
 [share, '', 'outweigh', advantage, ''],
 [share, '', 'say', import

In [26]:
e2e_relationship_clean = []

for rel in e2e_relationship:
    if (rel[1] != '') and (rel[-1] != ''):
        e2e_relationship_clean.append(rel)

print(e2e_relationship_clean)

[[Japan, 'GPE', 'allege', Kong, 'GPE'], [Indonesia, 'GPE', 'make', Malaysia, 'GPE'], [PORTS, 'ORG', 'hit', CPO, 'ORG'], [LANKA, 'GPE', 'approve', Agriculture, 'ORG'], [BANK, 'ORG', 'merge', yen, 'MONEY'], [Komatsu, 'PERSON', 'tell', Reuters, 'ORG'], [Sumitomo, 'ORG', 'be', Sogoit, 'PERSON'], [Reuters, 'ORG', 'clarify', Monday, 'DATE'], [Indonesia, 'GPE', 'say', Monday, 'DATE'], [Bond, 'ORG', 'say', Cebu, 'GPE'], [Atlas, 'PERSON', 'tell', Kong, 'GPE'], [Corp, 'ORG', 'consider', Philippines, 'GPE'], [US, 'GPE', 'demand', Japan, 'GPE'], [US, 'GPE', 'remain', Brazil, 'GPE'], [Japan, 'GPE', 'drop', Brazil, 'GPE'], [Japan, 'GPE', 'ask', US, 'GPE'], [Keidanren, 'PERSON', 'remove', Kato, 'PERSON'], [AMATIL, 'ORG', 'propose', two, 'CARDINAL'], [US, 'GPE', 'revalue', Washington, 'GPE'], [US, 'GPE', 'want', Washington, 'GPE'], [Korea, 'GPE', 'cut', US, 'GPE'], [Gold, 'PERSON', 'combine', Australia, 'GPE'], [Britain, 'GPE', 'communicate', todayit, 'ORG'], [Japan, 'GPE', 'convince', US, 'GPE'], [Ja

In [ ]:
print(set(all_verbs))

In [33]:
df = pd.DataFrame(e2e_relationship_clean, columns=["subj", "subj_label", "verb", "obj", "obj_label"])

In [35]:
df_id = pd.read_csv('../database/entities.csv')

In [74]:
subj_idx = []
obj_idx = []

for idx, row in df.iterrows():
    subj = row['subj'].text.lower()
    obj = row['obj'].text.lower()

    idx = df_id.loc[df_id.name == subj]['id'].values
    try:
        idx = idx[0]
    except IndexError:
        idx = np.nan
        
    subj_idx.append(idx)

    idx = df_id.loc[df_id.name == obj]['id'].values
    try:
        idx = idx[0]
    except IndexError:
        idx = np.nan
    obj_idx.append(idx)

In [78]:
df['subj_id'] = subj_idx
df['obj_id'] = obj_idx

In [87]:
df.to_csv('../database/relationships.csv')

In [47]:
df_id

,Unnamed: 0,id,name,label,mentions
0,0,1,iran-iraq war,EVENT,5
1,1,2,the gulf war,EVENT,8
2,2,3,gulf war,EVENT,2
3,3,4,yom kippur war,EVENT,1
4,4,5,september m3,EVENT,2
...,...,...,...,...,...
24300,24300,24301,endaka,WORK_OF_ART,2
24301,24301,24302,superfund,WORK_OF_ART,1
24302,24302,24303,fort knox,WORK_OF_ART,1
24303,24303,24304,130p,WORK_OF_ART,1


In [35]:
similarity_matrix = [[] for x in range(len(set(all_verbs)))]

for idx, token1 in enumerate(set(all_verbs)):
    print(idx/len(set(all_verbs)), end="\r")
    for token2 in set(all_verbs):
        similarity_matrix[idx].append(token1.similarity(token2))


C:\Users\aubin\AppData\Local\Temp\ipykernel_16532\3095854996.py:6: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_matrix[idx].append(token1.similarity(token2))


KeyboardInterrupt: 

In [39]:
set(all_verbs)

{adapt,
 be,
 simplify,
 compare,
 import,
 account,
 employ,
 grow,
 see,
 say,
 lead,
 gather,
 agree,
 raise,
 maintain,
 say,
 have,
 expand,
 rise,
 compare,
 fail,
 indicate,
 rise,
 suffer,
 affect,
 decrease,
 complete,
 expect,
 watch,
 delay,
 say,
 say,
 make,
 refrain,
 think,
 be,
 act,
 sell,
 withstand,
 bring,
 expect,
 discuss,
 say,
 prompt,
 tap,
 collapse,
 store,
 place,
 say,
 dictate,
 hold,
 produce,
 combine,
 operate,
 comprise,
 say,
 include,
 note,
 dlrs,
 be,
 say,
 act,
 be,
 make,
 boost,
 occur,
 say,
 say,
 show,
 place,
 be,
 become,
 say,
 tell,
 fall,
 file,
 follow,
 force,
 create,
 say,
 be,
 sell,
 persist,
 say,
 have,
 say,
 receive,
 reduce,
 repeal,
 set,
 change,
 meet,
 slip,
 say,
 operate,
 be,
 fuel,
 approve,
 invest,
 include,
 have,
 pay,
 put,
 change,
 go,
 buy,
 like,
 lend,
 continue,
 represent,
 dlrs,
 draft,
 agree,
 sell,
 begin,
 begin,
 fit,
 fill,
 say,
 forecast,
 bothersomeonce,
 say,
 enjoin,
 be,
 be,
 ommitte,
 envisa

In [36]:
all_verbs_txt = [verb.text for verb in all_verbs]

In [37]:
filehandler = open("../stored_data/all_verbs.obj","wb")
pickle.dump(all_verbs_txt, filehandler)
filehandler.close()